# [모듈 2.1] 카탈로그 포트폴리오 및 프러덕트 배포
# 설치 코드 리포
- https://github.com/aws-samples/amazon-sagemaker-reusable-components/blob/main/package-cfn.md

---
## 필수 권한
- S3FulllAccess
- CloudFullAccess
- IAMCreaterole
- CodeBuildFull
- LambdaFull
- ServiceCatalogFull

In [1]:
%store -r

# 1. 카탈로그 프트폴리오 및 프더덕트 CF 확인

## 1.1 실행할 CF YAML 파일 다운로드 

In [2]:
! aws s3 ls {bucket}/amazon-sagemaker-reusable-components/sm-project-sc-portfolio.yaml 
! aws s3 cp s3://{bucket}/amazon-sagemaker-reusable-components/sm-project-sc-portfolio.yaml .
# ! aws s3 ls https://s3.{bucket_region}.amazonaws.com/{bucket}/amazon-sagemaker-reusable-components/sm-project-sc-portfolio.yaml 

2022-02-04 14:16:38       7210 sm-project-sc-portfolio.yaml
download: s3://sagemaker-ap-southeast-2-189546603447/amazon-sagemaker-reusable-components/sm-project-sc-portfolio.yaml to ./sm-project-sc-portfolio.yaml


## 1.2 실행 결과의 Output 확인
- PortfolioId
    - 서비스 카탈로그 데이터 과학 포트폴리오 ID
- ProductId
    - 서비스 카탈로그 데이터 과학 제품 ID
- ProductName
    - 서비스 카탈로그 데이터 과학 제품 이름
- ProvisioningArtifactIds
    - 서비스 카탈로그 데이터 과학 프로비저닝 아티팩트 ID
- ProvisioningArtifactNames
    - 서비스 카탈로그 데이터 과학 프로비저닝 아티팩트 이름
- FSIngestionProductPolicyArn
    - Feature Store 수집 제품을 시작하기 위해 AmazonSageMakerServiceCatalogProductsLaunchRole에 대한 관리형 정책      
- AmazonSageMakerExecutionRolePolicyArn
    - Feature Store 수집 실험으로 노트북을 실행하려면 권한이 있는 Amazon SageMaker 실행 역할에 대한 관리형 정책      
- AmazonSageMakerExecutionRoleName
    - Amazon SageMaker 실행 역할의 이름

In [3]:
!pygmentize "sm-project-sc-portfolio.yaml"   | sed -n 6,50p

Outputs:
  ProductId:
    Description: Service Catalog data science product Id
    Value:
      Ref: FeatureStoreDataIngestionProduct
  PortfolioId:
    Description: Service Catalog data science portfolio Id
    Value:
      Ref: DataScienceAutomationPortfolio
  ProductName:
    Description: Service Catalog data science product name
    Value:
      Fn::GetAtt:
      - FeatureStoreDataIngestionProduct
      - ProductName
  ProvisioningArtifactIds:
    Description: Service Catalog data science provisioning artifact Ids
    Value:
      Fn::GetAtt:
      - FeatureStoreDataIngestionProduct
      - ProvisioningArtifactIds
  ProvisioningArtifactNames:
    Description: Service Catalog data science provisioning artifact names
    Value:
      Fn::GetAtt:
      - FeatureStoreDataIngestionProduct
      - ProvisioningArtifactNames
  FSIngestionProductPolicyArn:
    Description: Managed policy for AmazonSageMakerServiceCatalogProductsLaunchRole
      to launch an Feature Store ingestion product
 

## 1.2 CF 파라미터
- SCPortfolioPrincipalRoleArn
    - 서비스 카탈로그 제품에 대한 액세스 권한이 부여될 IAM 역할
- SCProductLaunchRoleArn
    - SageMaker Studio가 제품을 시작될 때 서비스 카탈로그가 맡는 IAM 역할

In [4]:
!pygmentize "sm-project-sc-portfolio.yaml"   | sed -n 51,59p

Parameters:
  SCPortfolioPrincipalRoleArn:
    Type: String
    Description: IAM role which will be granted access to Service Catalog products
  SCProductLaunchRoleArn:
    Type: String
    Description: IAM role that Service Catalog assumes when SageMaker Studio launches
      a product
    Default: AmazonSageMakerServiceCatalogProductsLaunchRole


## 1.2 CF 생성될 리소스
- DataScienceAutomationPortfolio
    - 이 포트폴리오는 SageMaker Studio용 재사용 가능한 데이터 과학 자동화 구성 요소 모음입니다.
- FeatureStoreDataIngestionProduct
    - 이 템플릿은 데이터 변환을 하는 Data Wrangler 및 SageMaker 파이프라인을 사용하여 S3 버킷에서 피쳐 그룹으로 데이터 수집을 자동화하기 위한 SageMaker 프로젝트를 생성합니다.


In [5]:
!pygmentize "sm-project-sc-portfolio.yaml"   | sed -n 65,89p

Resources:
  DataScienceAutomationPortfolio:
    Type: AWS::ServiceCatalog::Portfolio
    Properties:
      ProviderName: Data Science Administration Team
      Description: This portfolio is a collection of re-usable data science automation
        components for SageMaker Studio
      DisplayName: Re-usable data science automation components for your ML environment
  FeatureStoreDataIngestionProduct:
    Type: AWS::ServiceCatalog::CloudFormationProduct
    Properties:
      Name: Automated Feature Transformation and Ingestion Pipeline v1.0
      Description: This template creates a SageMaker project for automating a data
        ingestion from an S3 bucket into a feature group using Data Wrangler data
        transformation and SageMaker Pipelines
      Owner: Data Science Administration Team
      ProvisioningArtifactParameters:
      - Name: Automated Feature Transformation and Ingestion Pipeline v1.0
        Description: SageMaker project to transform and ingest features into a fe

## 1.3 이외 CF 생성될 리소스
- SCPortfolioFeatureStoreDataIngestionProductAssociation
- SCPortfolioPrincipleAssociation
- SCFeatureStoreDataIngestionProductLaunchRoleConstraint
- AmazonSageMakerExecutionRolePolicy
- AmazonSageMakerServiceCatalogFSIngestionProductPolicy


In [6]:
!pygmentize "sm-project-sc-portfolio.yaml"   | sed -n 90,199p

  SCPortfolioFeatureStoreDataIngestionProductAssociation:
    Type: AWS::ServiceCatalog::PortfolioProductAssociation
    Properties:
      PortfolioId:
        Ref: DataScienceAutomationPortfolio
      ProductId:
        Ref: FeatureStoreDataIngestionProduct
  SCPortfolioPrincipleAssociation:
    Type: AWS::ServiceCatalog::PortfolioPrincipalAssociation
    Properties:
      PortfolioId:
        Ref: DataScienceAutomationPortfolio
      PrincipalARN:
        Ref: SCPortfolioPrincipalRoleArn
      PrincipalType: IAM
  SCFeatureStoreDataIngestionProductLaunchRoleConstraint:
    Type: AWS::ServiceCatalog::LaunchRoleConstraint
    DependsOn:
    - SCPortfolioPrincipleAssociation
    Properties:
      Description:
        Fn::Sub: AWS Service Catalog uses ${SCProductLaunchRoleArn} to launch SageMaker
          projects
      PortfolioId:
        Ref: DataScienceAutomationPortfolio
      ProductId:
        Ref: FeatureStoreDataIngestionProduct
      RoleArn:
        Fn::If:
        - DefaultL

In [7]:
! aws s3 cp sm-project-sc-portfolio.yaml  s3://{bucket}/amazon-sagemaker-reusable-components/sm-project-sc-portfolio.yaml 

upload: ./sm-project-sc-portfolio.yaml to s3://sagemaker-ap-southeast-2-189546603447/amazon-sagemaker-reusable-components/sm-project-sc-portfolio.yaml


# 2. AWS Service Catalog 제품 포트폴리오 배포
- 'sm-project-sc-portfolio.yaml' 를 실행하여 카탈로그 포트폴리오 및 프로덕트에 등록하는 과정
- 아래와 같은 주요 파라미터가 필요 합니다.
```
aws cloudformation create-stack \
    --template-url # 포트폴리오 및 프로덕트가 정의된 CF YAML
    --region # 리젼
    --stack-name # 스텍 이름
    --disable-rollback 
    --capabilities CAPABILITY_NAMED_IAM \
    --parameters \
        ParameterKey=SCPortfolioPrincipalRoleArn,ParameterValue= <> # SageMaker Execution Role 할당
```

In [8]:
import sagemaker
region = sagemaker.Session().boto_region_name
SC_PORTFOLIO_STACK_NAME='sm-project-sc-portfolio'

In [9]:
%%sh -s "{bucket}" "{SC_PORTFOLIO_STACK_NAME}" "{region}"

bucket_name=$1
sc_portfolio_stack_name=$2
region=$3
echo 'bucket_name' :  $bucket_name
echo 'stack_name' :  $sc_portfolio_stack_name
echo 'Bucket Region' : $region
    
export SM_DOMAIN_ID=$(aws sagemaker list-domains \
    --output text --query 'Domains[0].DomainId')
echo 'SageMaker Studio Domain ID' : $SM_DOMAIN_ID

export SM_EXECUTION_ROLE=$(aws sagemaker describe-domain \
    --domain-id $SM_DOMAIN_ID \
    --output text --query 'DefaultUserSettings.ExecutionRole')
echo 'SageMaker Execution Role' : $SM_EXECUTION_ROLE

echo 'CF File: ' : https://s3.$region.amazonaws.com/$bucket_name/amazon-sagemaker-reusable-components/sm-project-sc-portfolio.yaml

aws cloudformation create-stack \
    --template-url https://s3.$region.amazonaws.com/$bucket_name/amazon-sagemaker-reusable-components/sm-project-sc-portfolio.yaml \
    --region $region \
    --stack-name $sc_portfolio_stack_name  \
    --disable-rollback \
    --capabilities CAPABILITY_NAMED_IAM \
    --parameters \
        ParameterKey=SCPortfolioPrincipalRoleArn,ParameterValue=$SM_EXECUTION_ROLE

bucket_name : sagemaker-ap-southeast-2-189546603447
stack_name : sm-project-sc-portfolio
Bucket Region : ap-southeast-2
SageMaker Studio Domain ID : d-bqnj3arguz1d
SageMaker Execution Role : arn:aws:iam::189546603447:role/MLOps-custom-template-gonsoomoon
CF File:  : https://s3.ap-southeast-2.amazonaws.com/sagemaker-ap-southeast-2-189546603447/amazon-sagemaker-reusable-components/sm-project-sc-portfolio.yaml
{
    "StackId": "arn:aws:cloudformation:ap-southeast-2:189546603447:stack/sm-project-sc-portfolio/082db950-8651-11ec-a081-0236e46e2510"
}


## 2.1 클라우드 포메이션 진행 확인

- 클라우드 포메이션의 콘솔로 가셔서 sm-project-sc-portfolio 가 성공적으로 끝났는지를 확인 합니다.

![cf_deploy_portfolio.png](img2/cf_deploy_portfolio.png)

## 2.2 서비스 카탈로그 및 프러덕트 생성 확인
- 서비스 카탈로그 콘솔로 이동하여 아래 처럼 생성이 되었는지를 확인 합니다.

![service_catalog.png](img2/service_catalog.png)

# 3. 서비스 카탈로그 런치 및 SageMaker 실행 IAM 역할에 권한 추가

- AmazonSageMakerServiceCatalogProductsLaunchRole 에 권한 추가
    - AWS Service Catalog는 기본 [`AmazonSageMakerServiceCatalogProductsLaunchRole` IAM 역할](https://docs.aws.amazon.com/sagemaker/latest/dg/security-iam-awsmanpol-sc.html)을 사용하여 SageMaker 프로젝트와 함께 CloudFormation 템플릿을 시작합니다. 이 역할은 Studio 사용자에 대해 SageMaker 프로젝트를 활성화한 경우 SageMaker Studio를 프로비저닝하는 동안 자동으로 생성됩니다.
    - Feature Store 수집 제품을 SageMaker 프로젝트로 배포하려면 이 역할에 추가 권한이 필요합니다. 필요한 모든 권한은 [관리형 정책 리소스 `AmazonSageMakerServiceCatalogFSIngestionProductPolicy`](cfn-templates/sm-project-sc-portfolio.yaml)에 정의되어 있으며 이를 역할(`AmazonSageMakerServiceCatalogProductsLaunchRole`) 에 첨부해야 합니다.
    - 이는  SageMaker 프로젝트 배포를 시작하기 전에 해야 합니다.


- AmazonSageMakerExecutionRole 에 권한 추가
    - 예를 들어 [CloudFormation API](https://docs.aws.amazon.com/AWSCloudFormation/latest/APIReference/API_Operations.html)를 호출하여 제공된 노트북에서 일부 코드 셀을 실행하려면 SageMaker 실행 역할에 추가 권한이 필요합니다. 이러한 권한은 [관리형 정책 리소스 `AmazonSageMakerExecutionRolePolicy`](cfn-templates/sm-project-sc-portfolio.yaml)에 정의되어 있으며 SageMaker 실행 역할에 연결되어야 합니다.

다음 명령을 실행하여 생성된 관리형 정책을 `AmazonSageMakerServiceCatalogProductsLaunchRole` 및 SageMaker 실행 IAM 역할에 연결합니다.




In [ ]:
%%sh -s "{SC_PORTFOLIO_STACK_NAME}" 

sc_portfolio_stack_name=$1
export SM_SC_FS_INGESTION_POLICY_ARN=$(aws cloudformation describe-stacks \
    --stack-name $sc_portfolio_stack_name \
    --output text \
    --query 'Stacks[0].Outputs[?OutputKey==`FSIngestionProductPolicyArn`].OutputValue')

echo 'SM_SC_FS_INGESTION_POLICY_ARN' : $SM_SC_FS_INGESTION_POLICY_ARN
    
export SM_EXECUTION_ROLE_POLICY_ARN=$(aws cloudformation describe-stacks \
    --stack-name $sc_portfolio_stack_name \
    --output text \
    --query 'Stacks[0].Outputs[?OutputKey==`AmazonSageMakerExecutionRolePolicyArn`].OutputValue')

echo ''
echo 'SM_EXECUTION_ROLE_POLICY_ARN' : $SM_EXECUTION_ROLE_POLICY_ARN    
    
export SM_EXECUTION_ROLE_NAME=$(aws cloudformation describe-stacks \
    --stack-name $sc_portfolio_stack_name \
    --output text \
    --query 'Stacks[0].Outputs[?OutputKey==`AmazonSageMakerExecutionRoleName`].OutputValue')

echo ''
echo 'SM_EXECUTION_ROLE_NAME' : $SM_EXECUTION_ROLE_NAME    

aws iam attach-role-policy \
    --role-name AmazonSageMakerServiceCatalogProductsLaunchRole \
    --policy-arn $SM_SC_FS_INGESTION_POLICY_ARN

aws iam attach-role-policy \
    --role-name  $SM_EXECUTION_ROLE_NAME \
    --policy-arn $SM_EXECUTION_ROLE_POLICY_ARN

# 4. 노트북 끝
- README 파일로 이동하시고, 아래는 트러블 슈팅시 참고 하세요.

# A. 트러블 슈팅 커맨드

## A.1 클라우드 포메이션 스택 진행 상황, 삭제 및 기타 명령어

In [14]:
# 스택 이벤트 실행 내역 보여주기
! aws cloudformation describe-stack-events --stack-name amazon-sagemaker-reusable-components-package-cfn

{
    "StackEvents": [
        {
            "StackId": "arn:aws:cloudformation:ap-southeast-2:057716757052:stack/amazon-sagemaker-reusable-components-package-cfn/d112c110-7929-11ec-9477-0275392b8e7c",
            "EventId": "0cee1680-792a-11ec-83ed-0abbffeda692",
            "StackName": "amazon-sagemaker-reusable-components-package-cfn",
            "LogicalResourceId": "amazon-sagemaker-reusable-components-package-cfn",
            "PhysicalResourceId": "arn:aws:cloudformation:ap-southeast-2:057716757052:stack/amazon-sagemaker-reusable-components-package-cfn/d112c110-7929-11ec-9477-0275392b8e7c",
            "ResourceType": "AWS::CloudFormation::Stack",
            "Timestamp": "2022-01-19T13:16:45.402Z",
            "ResourceStatus": "ROLLBACK_FAILED",
            "ResourceStatusReason": "The following resource(s) failed to delete: [StartBuildLambda]. "
        },
        {
            "StackId": "arn:aws:cloudformation:ap-southeast-2:057716757052:stack/amazon-sagemaker-reusable-co

In [98]:
# 스택 삭제
! aws cloudformation delete-stack --stack-name amazon-sagemaker-reusable-components-package-cfn

## A.2 S3 CLI 관련 명령어

In [53]:
! aws s3 ls https://s3.{bucket_region}.amazonaws.com/{bucket}/amazon-sagemaker-reusable-components/sm-project-sc-portfolio.yaml
# ! aws s3 ls https://s3.ap-southeast-2.amazonaws.com/sagemaker-ap-southeast-2-057716757052/amazon-sagemaker-reusable-components/sm-project-sc-portfolio.yaml


Error parsing parameter 'paths': Unable to retrieve https://s3.ap-northeast-2.amazonaws.com/mlops-gonsoomoon-share/amazon-sagemaker-reusable-components/sm-project-sc-portfolio.yaml: received non 200 status code of 403


In [97]:
! aws s3 ls {bucket} --recursive

2022-01-23 10:42:21      22207 amazon-sagemaker-reusable-components/amazon-sagemaker-reusable-components.zip
2022-01-23 10:42:23      15589 amazon-sagemaker-reusable-components/project-s3-fs-ingestion.yaml
2022-01-23 10:42:22        606 amazon-sagemaker-reusable-components/seed-code/s3-fs-ingestion-v1.0.zip
2022-01-23 10:42:23       7196 amazon-sagemaker-reusable-components/sm-project-sc-portfolio.yaml


In [84]:
! aws s3 rm s3://{bucket} --recursive

delete: s3://sagemaker-ap-southeast-2-057716757052/amazon-sagemaker-reusable-components/amazon-sagemaker-reusable-components.zip
delete: s3://sagemaker-ap-southeast-2-057716757052/amazon-sagemaker-reusable-components/project-s3-fs-ingestion.yaml
delete: s3://sagemaker-ap-southeast-2-057716757052/amazon-sagemaker-reusable-components/seed-code/s3-fs-ingestion-v1.0.zip
delete: s3://sagemaker-ap-southeast-2-057716757052/amazon-sagemaker-reusable-components/sm-project-sc-portfolio.yaml


# B. 에러 케이스



## B.1 sm-project-sc-portfolio.yaml 스택 실행시 에러
아래 스택 실행시에 에러 발생 함. 

Template error: Fn::Select cannot select nonexistent value at index 2

```
AmazonSageMakerExecutionRoleName:
    Description: Name of the Amazon SageMaker execution role
    Value:
      Fn::Select:
      - 1 # 기존에 2 였음. 1로 변경 함
```
아래 파일을 다음과 같이 변경 
amazon-sagemaker-reusable-components/build/sm-project-sc-portfolio.yaml
```
Before:
  AmazonSageMakerExecutionRoleName:
    Description: Name of the Amazon SageMaker execution role
    Value: !Select [2, !Split ['/', !Ref SCPortfolioPrincipalRoleArn ] ] 
```
```
After: 
  AmazonSageMakerExecutionRoleName:
    Description: Name of the Amazon SageMaker execution role
    Value: !Select [1, !Split ['/', !Ref SCPortfolioPrincipalRoleArn ] ] 


```

## B.2 에러: seed-code/s3-fs-ingestion-v1.0.zip 생성시 전부 아티펙트를 포함하지 않음.
```
Zipping MLOps project seed code
147	  adding: .ipynb_checkpoints/ (stored 0%)
148	  adding: .ipynb_checkpoints/buildspec-checkpoint.yml (deflated 54%)
149	  adding: buildspec.yml (deflated 54%)
```
정상 케이스
```
Zipping MLOps project seed code
  adding: LICENSE (deflated 39%)
  adding: pipeline/ (stored 0%)
  adding: pipeline/pipeline.py (deflated 70%)
  adding: buildspec.yml (deflated 54%)
  adding: .ipynb_checkpoints/ (stored 0%)
  adding: .ipynb_checkpoints/buildspec-checkpoint.yml (deflated 54%)
  adding: build.py (deflated 63%)
  adding: README.md (stored 0%)
  adding: functions/ (stored 0%)
  adding: functions/start_fs_ingestion.py (deflated 58%)
```

### 해결
- Before
    - zip -r amazon-sagemaker-reusable-components.zip . -i "*.yaml" "*.yml" "*.sh"
- After
    - zip -r amazon-sagemaker-reusable-components.zip . -i "*.yaml" "*.yml" "*.sh" "*.py"

## B.3. 포트폴리오 및 프러덕트 배포시 에러
- 서율리젼에서 진행싱 ProductLaunchRoleConstraint 생성시에 에러 발생. 원인을 찾지 못함. 
    - 시드니 리젼에서 실행하니 에러 발생 하지 않음. 
    - 조금 더 원인 파악이 필요 함.


![error-deploy-portfolio-product.png](img2/error-deploy-portfolio-product.png)